In [20]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [21]:
#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x
      
    def put_data(self, item):
        self.data.append(item)
        
    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R
            loss.backward()
        self.optimizer.step()
        self.data = []

In [23]:
def main():
    env = gym.make('CartPole-v1')
    pi = Policy().to(device)
    score = 0.0
    print_interval = 20
    
    for n_epi in range(10000):
        s, _ = env.reset()
        s = torch.from_numpy(s).float().to(device)
        done = False
        
        while not done: # CartPole-v1 forced to terminates at 500 step.
            prob = pi(s)
            m = Categorical(prob)
            a = m.sample()
            s_prime, r, done, truncated, info = env.step(a.item())
            s_prime = torch.from_numpy(s_prime).float().to(device)
            pi.put_data((r,prob[a]))
            s = s_prime
            score += r
            
        pi.train_net()
        
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()

In [24]:
if __name__ == '__main__':
    print(device)
    main()

# of episode :20, avg score : 22.3
# of episode :40, avg score : 21.9
# of episode :60, avg score : 24.7
# of episode :80, avg score : 23.4
# of episode :100, avg score : 27.25
# of episode :120, avg score : 23.45
# of episode :140, avg score : 22.9
# of episode :160, avg score : 23.35
# of episode :180, avg score : 24.35
# of episode :200, avg score : 33.45
# of episode :220, avg score : 27.05
# of episode :240, avg score : 27.0
# of episode :260, avg score : 27.8
# of episode :280, avg score : 31.35
# of episode :300, avg score : 29.55
# of episode :320, avg score : 33.45
# of episode :340, avg score : 36.0
# of episode :360, avg score : 33.15
# of episode :380, avg score : 37.4
# of episode :400, avg score : 39.9
# of episode :420, avg score : 39.45
# of episode :440, avg score : 39.65
# of episode :460, avg score : 35.35
# of episode :480, avg score : 32.7
# of episode :500, avg score : 42.2
# of episode :520, avg score : 38.5
# of episode :540, avg score : 39.55
# of episode :560,

KeyboardInterrupt: 